# IMDB Scrapper

In [67]:
# importing modules
import requests
from bs4 import BeautifulSoup
from lxml import etree as et
import time
import random
import json
from unidecode import unidecode
import lxml
import pandas as pd
import numpy as np

In [2]:
start_url = "https://www.imdb.com"
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}
# movie_urls = []

In [36]:
response = requests.get(start_url, headers=header)
soup = BeautifulSoup(response.text, "lxml")
dom  = et.HTML(str(soup))

In [37]:
images = soup.find_all('img')
anchor_tags = soup.find_all('a')

image_list = [image['src'] for image in images if image['src'].endswith(('jpg', 'jpeg', 'png'))]
anchor_tag_list = []

for a in anchor_tags:
    if a['href'][0:6]!='https':
        a['href'] = "https://www.imdb.com" + a['href']
    anchor_tag_list.append(a['href'])


In [45]:
# print(image_list)
# print(anchor_tag_list)

In [46]:
df = pd.DataFrame(columns=['image_src', 'anchor_tag_links'])

if len(anchor_tag_list)>len(image_list):
    for _ in range(len(image_list), len(anchor_tag_list)):
        image_list.append(None)
elif len(image_list)>len(anchor_tag_list):
    for _ in range(len(anchor_tag_list),len(image_list)):
        anchor_tag_list.append(None)
        
df['anchor_tag_links'] = anchor_tag_list
df['image_src'] = image_list
df


,image_src,anchor_tag_links
0,https://m.media-amazon.com/images/M/MV5BMjM2MT...,https://www.imdb.com/?ref_=nv_home
1,https://m.media-amazon.com/images/M/MV5BNzgxZT...,https://www.imdb.comhttps://www.imdb.com/calen...
2,https://m.media-amazon.com/images/M/MV5BMDA3Mm...,https://www.imdb.com/chart/top/?ref_=nv_mv_250
3,https://m.media-amazon.com/images/M/MV5BMDdiMj...,https://www.imdb.com/chart/moviemeter/?ref_=nv...
4,https://m.media-amazon.com/images/M/MV5BOWE2Zm...,https://www.imdb.com/feature/genre/?ref_=nv_ch_gr
...,...,...
115,None,https://www.imdb.comhttps://www.amazon.jobs/en...
116,None,https://www.imdb.com/conditions?ref_=ft_cou
117,None,https://www.imdb.com/privacy?ref_=ft_pvc
118,None,https://www.imdb.com/privacy/adpreferences/


In [47]:
# Saving the csv file
df.to_csv('data/imdb_links.csv')

## Proposed Solution

- IMDB is a Movie and TV Series recommendation website. 
- Users mainly use IMDB for searching for their next tv series they want to watch, check the movie reviwes and rating. 
- Therefore, I am proposing to extract the movie and tv series details those are comes in top 250 or currently trending
- For this I am going to write a function in which you can scrape the details of the movies like their {title, year_relased, rating, top cast, movie_url, genre, director_name}


In [131]:
def imdb_scrapper(source_url, output_file_name, save_file=False):
    start_url = source_url
    header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}
    response = requests.get(start_url, headers=header)
    soup = BeautifulSoup(response.text, "lxml")
    dom  = et.HTML(str(soup))
    movie_urls_list = dom.xpath('//td[@class="titleColumn"]/a/@href')
    movie_urls = []
    df = pd.DataFrame(columns=['rank', 'movie_name', 'movie_url', 'movie_year', 'genre', 'director_name', 'rating', 'cast_and_crew'])

    for i in movie_urls_list:
        long_url = "https://www.imdb.com" + i
        short_url = long_url.split("?")[0]
        movie_urls.append(short_url)

    def time_delay():
        time.sleep(random.randint(2, 4))
    
    rank_list = np.array([])
    movie_name_list = np.array([])
    movie_year_list = np.array([])
    genre_list = np.array([])
    director_name_list = np.array([])
    rating_list = np.array([])
    all_actors_list = []

    for movie_url in movie_urls:
        
        response = requests.get(movie_url, headers=header)
        soup = BeautifulSoup(response.content, 'html.parser')
        dom = et.HTML(str(soup))
        try:
            rank = movie_urls.index(movie_url) + 1
            movie_name = dom.xpath('//h1[@data-testid="hero-title-block__title"]/text()')[0]
            movie_year = dom.xpath('//a[@class="ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-f26752fb-1 hMnkBf"]/text()')[0]
            genre = dom.xpath('//span[@class="ipc-chip__text"]/text()')[0]
            director_name = dom.xpath('//a[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]/text()')[0]
            rating = dom.xpath('//span[@class="sc-7ab21ed2-1 eUYAaq"]/text()')[0]
            actors_list = dom.xpath('//a[@data-testid="title-cast-item__actor"]/text()')
            actors_list = np.array([ unidecode(i) for i in actors_list])
        except:
            print("AttributeError:" + movie_url)
    
        # df.append(pd.Series({'rank': rank,
        #            'movie_name': movie_name,
        #            'movie_url': movie_url,
        #            'movie_year': movie_year,
        #            'genre': genre,
        #            'director_name': director_name,
        #            'rating': rating,
        #            'cast_and_crew': actors_list}, df.columns), ignore_index=True)
        
        rank_list = np.append(rank_list, np.array(rank))
        movie_name_list = np.append(movie_name_list, np.array(movie_name))
        movie_year_list = np.append(movie_year_list, np.array(movie_year))
        genre_list= np.append(genre_list, np.array(genre))
        director_name_list= np.append(director_name_list, np.array(director_name))
        rating_list = np.append(rating_list, np.array(rating))
        all_actors_list.append(actors_list)

        # time_delay()
    
    df['rank'] = rank_list
    df['movie_name'] = movie_name_list
    df['movie_url'] = movie_urls
    df['movie_year'] = movie_year_list
    df['genre'] = genre_list
    df['director_name'] = director_name_list
    df['rating'] = rating_list
    df['cast_and_crew'] = all_actors_list
    
    if save_file:
        df.to_csv('data/'+ output_file_name)
    return df

### Top 250 Movies

In [125]:
top_250_movies = imdb_scrapper(source_url="https://www.imdb.com/chart/top/?ref_=nv_td_mv250", output_file_name='top250movies.csv', save_file=True)
top_250_movies.head()

,rank,movie_name,movie_url,movie_year,genre,director_name,rating,cast_and_crew
0,1.0,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,1994,Drama,Frank Darabont,9.3,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,2.0,The Godfather,https://www.imdb.com/title/tt0068646/,1972,Crime,Francis Ford Coppola,9.2,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,3.0,The Dark Knight,https://www.imdb.com/title/tt0468569/,2008,Action,Christopher Nolan,9.0,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,4.0,The Godfather: Part II,https://www.imdb.com/title/tt0071562/,1974,Crime,Francis Ford Coppola,9.0,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,5.0,12 Angry Men,https://www.imdb.com/title/tt0050083/,1957,Crime,Sidney Lumet,9.0,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
...,...,...,...,...,...,...,...,...
245,246.0,The Help,https://www.imdb.com/title/tt1454029/,2011,Drama,Tate Taylor,8.1,"[Viola Davis, Emma Stone, Octavia Spencer, Bry..."
246,247.0,Dersu Uzala,https://www.imdb.com/title/tt0071411/,1975,Adventure,Akira Kurosawa,8.2,"[Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov..."
247,248.0,Aladdin,https://www.imdb.com/title/tt0103639/,1992,Animation,Ron Clements,8.0,"[Scott Weinger, Robin Williams, Linda Larkin, ..."
248,249.0,Gandhi,https://www.imdb.com/title/tt0083987/,1982,Biography,Richard Attenborough,8.0,"[Ben Kingsley, John Gielgud, Rohini Hattangadi..."


### Top 250 TV Series

In [132]:
top_250_series = imdb_scrapper(source_url="https://www.imdb.com/chart/toptv/?ref_=nv_td_tv250", output_file_name='top250TVSeries.csv', save_file=True)
top_250_series.head()

AttributeError


,rank,movie_name,movie_url,movie_year,genre,director_name,rating,cast_and_crew
0,1.0,Planet Earth II,https://www.imdb.com/title/tt5491994/,2016,Documentary,David Attenborough,9.5,"[David Attenborough, Chadden Hunter, Gordon Bu..."
1,2.0,Breaking Bad,https://www.imdb.com/title/tt0903747/,2008–2013,Crime,Vince Gilligan,9.5,"[Bryan Cranston, Aaron Paul, Anna Gunn, Betsy ..."
2,3.0,Planet Earth,https://www.imdb.com/title/tt0795176/,2006,Documentary,Sigourney Weaver,9.4,"[Sigourney Weaver, David Attenborough, Nikolay..."
3,4.0,Band of Brothers,https://www.imdb.com/title/tt0185906/,2001,Drama,Scott Grimes,9.4,"[Scott Grimes, Damian Lewis, Ron Livingston, S..."
4,5.0,Chernobyl,https://www.imdb.com/title/tt7366338/,2019,Drama,Craig Mazin,9.4,"[Jessie Buckley, Jared Harris, Stellan Skarsga..."


### Most popular TV Shows

In [133]:

most_popular_tv_shows = imdb_scrapper(source_url="https://www.imdb.com/chart/tvmeter/?ref_=nv_td_mptv", output_file_name='mostPopularTVShows.csv', save_file=True)
most_popular_tv_shows.head()

AttributeError


,rank,movie_name,movie_url,movie_year,genre,director_name,rating,cast_and_crew
0,1.0,The Last of Us,https://www.imdb.com/title/tt3581920/,2023–,Action,Neil Druckmann,9.2,"[Pedro Pascal, Bella Ramsey, Gabriel Luna, Ann..."
1,2.0,Poker Face,https://www.imdb.com/title/tt14269590/,2023–,Crime,Rian Johnson,8.2,"[Natasha Lyonne, Benjamin Bratt, Elijah George..."
2,3.0,The White Lotus,https://www.imdb.com/title/tt13406094/,2021–2023,Comedy,Mike White,7.9,"[Jennifer Coolidge, Jon Gries, F. Murray Abrah..."
3,4.0,Lockwood & Co.,https://www.imdb.com/title/tt13802576/,2023–,Action,Ruby Stokes,7.5,"[Ruby Stokes, Cameron Chapman, Ali Hadji-Heshm..."
4,5.0,Yellowstone,https://www.imdb.com/title/tt4236770/,2018–2023,Drama,John Linson,8.7,"[Kevin Costner, Luke Grimes, Kelly Reilly, Wes..."


### Most Popular Movies

In [134]:
most_popular_movies = imdb_scrapper(source_url="https://www.imdb.com/chart/moviemeter/?ref_=nv_td_mpm", output_file_name='mostPopularMovies.csv', save_file=True)
most_popular_movies.head()

AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError


,rank,movie_name,movie_url,movie_year,genre,director_name,rating,cast_and_crew
0,1.0,You People,https://www.imdb.com/title/tt14826022/,2023,Comedy,Kenya Barris,5.5,"[Jonah Hill, Lauren London, Eddie Murphy, Juli..."
1,2.0,Babylon,https://www.imdb.com/title/tt10640346/,2022,Comedy,Damien Chazelle,7.4,"[Brad Pitt, Margot Robbie, Jean Smart, Olivia ..."
2,3.0,Black Panther: Wakanda Forever,https://www.imdb.com/title/tt9114286/,2022,Action,Ryan Coogler,6.9,"[Letitia Wright, Lupita Nyong'o, Danai Gurira,..."
3,4.0,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/,2022,Action,James Cameron,7.8,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
4,5.0,The Menu,https://www.imdb.com/title/tt9764362/,2022,Horror,Mark Mylod,7.2,"[Ralph Fiennes, Anya Taylor-Joy, Nicholas Houl..."


## Conclusion

- The above approach is slow because it is scrapping the data from 100 - 250 web pages and also appending it on the csv file.
- The scrapping is taking aroung 6-10 minutes for every function call.
- It's a better way of scrapping the imdb website data.